In [1]:
# Add the parent directory of the current working directory to the Python path at runtime. 
# In order to import modules from the src directory.
import os
import sys 

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)

In [7]:
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from src.stat_utils import *
from src.anl_utils import load_data, get_session_data

In [3]:
sim_results_folder = '../results/simulation'
data_folder = '../data'

sim_at_file = os.path.join(sim_results_folder, 'highres_arnold_tongues.npy')
emp_at_file = os.path.join(data_folder, 'Experiment.csv')

In [9]:
# Load the simulations results and the empirical data
sim_results = np.load(sim_at_file)[0].mean(axis=0)[::6, ::6]
sim_results_vector = sim_results.flatten()

data = load_data(emp_at_file)
# Get the data for the first session
data = get_session_data(data, 1)

# Map synchrony values to each condition in your DataFrame
data['Synchrony'] = data['Condition'].apply(lambda x: sim_results_vector[x-1])

In [10]:
# Define distribution and covariance structure for GEE
family = sm.families.Binomial()
covariance_structure = sm.cov_struct.Exchangeable()

# Fit the GEE model
model = smf.gee("Correct ~ Synchrony",
                    "SubjectID",
                    data,
                    cov_struct=covariance_structure,
                    family=family)
results = model.fit()

print(results.summary())

                               GEE Regression Results                              
Dep. Variable:                     Correct   No. Observations:                 6000
Model:                                 GEE   No. clusters:                        8
Method:                        Generalized   Min. cluster size:                 750
                      Estimating Equations   Max. cluster size:                 750
Family:                           Binomial   Mean cluster size:               750.0
Dependence structure:         Exchangeable   Num. iterations:                     6
Date:                     Wed, 05 Jun 2024   Scale:                           1.000
Covariance type:                    robust   Time:                         12:53:09
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1115      0.037      3.002      0.003       0.039       0.184
Synchro

In [11]:
print_wald_chi_square(results)

Wald Chi-Square:
+-----------+--------------------+-----------------------+
|  Variable |     Chi-Square     |        p-value        |
+-----------+--------------------+-----------------------+
| Intercept | 9.012156884475772  |  0.002681897414354512 |
| Synchrony | 20.923917102628227 | 4.778904663696652e-06 |
+-----------+--------------------+-----------------------+
